In [2]:
%load_ext autoreload
%autoreload 2

from utils.geo import GeoUtils
from pathlib import Path


## Data Cleaning

In [ ]:
def extract_address_from_url(url):
    """
    Extract and clean address from Domain.com.au URL format.
    
    This function handles both URL parsing and address cleaning to produce
    properly formatted addresses without needing further processing.
    
    Parameters:
    -----------
    url : str
        URL in format like '/4511-33-rose-lane-melbourne-vic-3000-16767655'
        
    Returns:
    --------
    string
        Cleaned address string in format '33 Rose Lane, Melbourne, VIC 3000'
    """
    
    if pd.isna(url) or not isinstance(url, str):
        return None
    
    # Remove leading slash if present
    url = url.lstrip('/')
    
    # Split by dashes
    parts = url.split('-')
    
    if len(parts) < 4:
        return None
    
    # Extract postcode (second to last part)
    postcode = parts[-2]
    
    # Extract state (third to last part)
    state = parts[-3]
    
    # Extract suburb (fourth to last part)
    suburb = parts[-4]
    
    # Everything before suburb is the street address
    street_parts = parts[:-4]
    
    # Skip unit number (first part if it's a digit) and join remaining parts
    if street_parts and street_parts[0].isdigit():
        street_parts = street_parts[1:]
    
    # Join street parts and replace dashes with spaces
    street_address = ' '.join(street_parts)
    
    # Clean the street address to remove unit numbers, apartment numbers, etc.
    cleaned_street = clean_street_address(street_address)
    
    # Create full address with cleaned street
    full_address = f"{cleaned_street}, {suburb.title()}, {state.upper()} {postcode}"
    
    return full_address


def clean_street_address(street_address):
    """
    Clean street address to keep only the last street number and street name.
    
    Removes unit numbers, apartment numbers, and other prefixes while preserving:
    - Only the LAST street number (including alphanumeric like "12a")
    - Street names
    - Street types (St, Road, Avenue, etc.)
    
    Parameters:
    -----------
    street_address : str
        Raw street address (e.g., "unit 1 47 53 wyndham st")
        
    Returns:
    --------
    str
        Cleaned street address (e.g., "53 Wyndham St")
    """
    
    if not street_address or not isinstance(street_address, str):
        return street_address
    
    # Split into words
    words = street_address.split()
    
    if not words:
        return street_address
    
    # Common prefixes to remove (case insensitive)
    prefixes_to_remove = {
        'unit', 'apt', 'apartment', 'suite', 'level', 'floor', 'shop', 'office',
        'rear', 'front', 'ground', 'basement', 'mezzanine', 'penthouse',
        'villa', 'townhouse', 'house', 'home', 'property', 'building'
    }
    
    # Remove prefixes from the beginning
    cleaned_words = []
    i = 0
    while i < len(words):
        word = words[i].lower()
        
        # Check if this word is a prefix to remove
        if word in prefixes_to_remove:
            # Skip this word and potentially the next one if it's a number
            i += 1
            # If the next word is a number, skip it too
            if i < len(words) and words[i].isdigit():
                i += 1
        else:
            # This is not a prefix, start collecting words
            break
    
    # Collect the remaining words
    cleaned_words = words[i:]
    
    if not cleaned_words:
        return street_address  # Return original if everything was removed
    
    # Find the street number and street name
    # We want the last numeric/alphanumeric word as the street number
    # and everything after it as the street name
    
    street_number = None
    street_name_start = 0
    
    # Look for the last numeric/alphanumeric word in the cleaned words
    for i in range(len(cleaned_words) - 1, -1, -1):
        word = cleaned_words[i]
        # Check if this word contains a number (could be "53", "12a", "415a")
        if any(char.isdigit() for char in word):
            street_number = word
            street_name_start = i + 1
            break
    
    # If no street number found, return the cleaned words as is (title case)
    if street_number is None:
        return ' '.join(word.title() for word in cleaned_words)
    
    # Extract street name (everything after the street number)
    street_name_words = cleaned_words[street_name_start:]
    
    if not street_name_words:
        # If no street name, just return the street number
        return street_number
    
    # Combine street number and street name
    street_name = ' '.join(word.title() for word in street_name_words)
    return f"{street_number} {street_name}"

# Test the function with some examples
test_urls = [
    '/4511-33-rose-lane-melbourne-vic-3000-16767655',
    '/10-speke-street-beaufort-vic-3373-16781407',
    '/36-390-burwood-highway-burwood-vic-3125-8592618',
    '/rear-70-neil-street-beaufort-vic-3373-15866775',
    # Test cases with unit numbers and complex addresses
    '/unit-1-47-53-wyndham-st-shepparton-vic-3630-12345678',
    '/415a-72-ascot-vale-road-flemington-vic-3031-87654321',
    '/apt-2-123-smith-street-south-melbourne-vic-3205-11223344'
]

print("Testing address extraction from url:")
print("=" * 70)
for url in test_urls:
    result = extract_address_from_url(url)
    print(f"URL: {url}")
    print(f"Address: {result}")
    print("-" * 70)
 


Testing address extraction:
Full Address: 33 rose lane, Melbourne, VIC 3000
Full Address: speke street, Beaufort, VIC 3373
Full Address: 390 burwood highway, Burwood, VIC 3125
Full Address: rear 70 neil street, Beaufort, VIC 3373


In [49]:

def clean_wayback_rental_data(input_data_dir, file_names, output_data_dir):
    """
    Clean and process rental listings summary data scraped from wayback.
    
    Parameters:
    -----------
    input_data_dir : Path
        Path to the directory containing the CSV files to preprocess
    file_names : list
        List of CSV file names to process
    output_data_dir : Path
        Path to the directory where cleaned data will be saved
        
    Returns:
    --------
    pd.DataFrame
        Cleaned and stacked dataframe with all rental listings
    """
    
    # Dictionary to store dataframes
    rental_data = {}
    
    # Load each CSV file
    for file_name in file_names:
        file_path = input_data_dir / file_name
        if file_path.exists():
            # Extract period from filename
            period = file_name.replace('rental_listings_', '').replace('.csv', '')
            
            print(f"Loading {file_name}...")
            df = pd.read_csv(file_path)
            
            # Add period column for reference
            df['period'] = period
            
            rental_data[period] = df
            print(f"  - Loaded {len(df):,} records")
        else:
            print(f"Warning: {file_name} not found!")
    
    print(f"\nSuccessfully loaded {len(rental_data)} datasets")
    
    # Stack all dataframes
    print("\nStacking dataframes...")
    stacked_df = pd.concat(rental_data.values(), ignore_index=True)
    print(f"Total records after stacking: {len(stacked_df):,}")
    
    # Display column info before cleaning
    print(f"\nColumns before cleaning: {list(stacked_df.columns)}")
    print(f"Data types before cleaning:")
    print(stacked_df.dtypes)
    
    # Convert numeric columns to appropriate types
    print("\nConverting numeric columns...")
    
    # Convert bedrooms, bathrooms, car_spaces to integer (handle NaN values)
    numeric_cols = ['bedrooms', 'bathrooms', 'car_spaces']
    for col in numeric_cols:
        if col in stacked_df.columns:
            # Convert to numeric, coercing errors to NaN
            stacked_df[col] = pd.to_numeric(stacked_df[col], errors='coerce')
            # Convert to nullable integer type
            stacked_df[col] = stacked_df[col].astype('Int64')
            print(f"  - {col}: converted to Int64")
    
    # Handle land_area: convert to numeric and replace 0 with NaN
    if 'land_area' in stacked_df.columns:
        stacked_df['land_area'] = pd.to_numeric(stacked_df['land_area'], errors='coerce')
        # Replace 0.0 with NaN
        stacked_df['land_area'] = stacked_df['land_area'].replace(0.0, np.nan)
        print(f"  - land_area: converted to float64, 0.0 values replaced with NaN")
    
    # Convert scraped_date to datetime
    if 'scraped_date' in stacked_df.columns:
        stacked_df['scraped_date'] = pd.to_datetime(stacked_df['scraped_date'], errors='coerce')
        print(f"  - scraped_date: converted to datetime")

    # Convert url to address string 
    stacked_df['address'] = stacked_df['url'].apply(extract_address_from_url)
    # Drop wayback_url and wayback_time columns
    columns_to_drop = ['wayback_url', 'wayback_time']
    existing_columns_to_drop = [col for col in columns_to_drop if col in stacked_df.columns]
    if existing_columns_to_drop:
        stacked_df = stacked_df.drop(columns=existing_columns_to_drop)
        print(f"  - Dropped columns: {existing_columns_to_drop}")

    
    # Remove duplicates, keeping the record with the earliest scraped_date
    print(f"\nRemoving duplicates...")
    print(f"Records before deduplication: {len(stacked_df):,}")
    
    # Sort by scraped_date to ensure we keep the earliest record
    stacked_df = stacked_df.sort_values('scraped_date', na_position='last')
    
    # Drop duplicates based on property_id, keeping the first (earliest) occurrence
    stacked_df = stacked_df.drop_duplicates(subset=['property_id'], keep='first')
    
    print(f"Records after deduplication: {len(stacked_df):,}")
    print(f"Duplicates removed: {len(rental_data) * len(stacked_df) - len(stacked_df):,}")

    # Geocode addresses
    geo_utils = GeoUtils()
    stacked_df['coordinates'] = stacked_df['address'].apply(geo_utils.geocode_nominatim)
    
    # Display final column info
    print(f"\nFinal data types:")
    print(stacked_df.dtypes)
    
    # Display summary statistics
    print(f"\nSummary statistics:")
    print(f"  - Total unique properties: {stacked_df['property_id'].nunique():,}")
    print(f"  - Date range: {stacked_df['scraped_date'].min()} to {stacked_df['scraped_date'].max()}")
    print(f"  - Unique suburbs: {stacked_df['suburb'].nunique():,}")
    
    # Create output directory if it doesn't exist
    output_data_dir.mkdir(parents=True, exist_ok=True)
    
    # Save the cleaned dataframe
    output_file = output_data_dir / "rental_listings_summary_cleaned.csv"
    print(f"\nSaving cleaned data to: {output_file}")
    stacked_df.to_csv(output_file, index=False)
    print(f"  - Saved {len(stacked_df):,} records to {output_file}")
    
    return stacked_df


In [57]:
# Apply the cleaning function
output_dir = Path('../data/processed/domain')
cleaned_df = clean_wayback_rental_data(data_dir, file_names, output_dir)


Loading rental_listings_2025_06.csv...
  - Loaded 3,382 records
Loading rental_listings_2025_03.csv...
  - Loaded 3,340 records
Loading rental_listings_2024_12.csv...
  - Loaded 1,562 records
Loading rental_listings_2024_09.csv...
  - Loaded 1,460 records
Loading rental_listings_2024_06.csv...
  - Loaded 1,786 records
Loading rental_listings_2024_03.csv...
  - Loaded 1,372 records
Loading rental_listings_2023_12.csv...
  - Loaded 179 records

Successfully loaded 7 datasets

Stacking dataframes...
Total records after stacking: 13,081

Columns before cleaning: ['property_id', 'url', 'rental_price', 'bedrooms', 'bathrooms', 'car_spaces', 'property_type', 'land_area', 'property_features', 'suburb', 'postcode', 'scraped_date', 'wayback_url', 'wayback_time', 'period']
Data types before cleaning:
property_id            int64
url                   object
rental_price          object
bedrooms             float64
bathrooms            float64
car_spaces           float64
property_type         obj

Loading in the saved intermediate result. Up till this stage, we haven't cleaned `rental_price` and some of the `address` (695 listings) failed to be geocoded.  

In [38]:
import pandas as pd
cleaned_df = pd.read_csv('../data/processed/domain/rental_listings_summary_cleaned.csv')
cleaned_df.head()

,property_id,url,rental_price,bedrooms,bathrooms,car_spaces,property_type,land_area,property_features,suburb,postcode,scraped_date,period,address,coordinates
0,16767655,/4511-33-rose-lane-melbourne-vic-3000-16767655,$750 Per Week,2.0,2.0,1.0,Apartment / Unit / Flat,NaN,"2, ,2, ,1,",MELBOURNE,3000,2023-12-02 19:11:26,2023_12,"33 rose lane, Melbourne, VIC 3000",POINT (144.9538211 -37.8151054)
1,16719394,/425-181-exhibition-street-melbourne-vic-3000-...,$600pw,2.0,1.0,1.0,Apartment / Unit / Flat,NaN,"2, ,1, ,1,",MELBOURNE,3000,2023-12-02 19:11:26,2023_12,"181 exhibition street, Melbourne, VIC 3000",POINT (144.968708 -37.8076807)
2,16760539,/901-225-elizabeth-street-melbourne-vic-3000-1...,$480 pw,1.0,1.0,1.0,Apartment / Unit / Flat,NaN,"1, ,1, ,1,",MELBOURNE,3000,2023-12-02 19:11:26,2023_12,"225 elizabeth street, Melbourne, VIC 3000",POINT (144.9844562 -37.7255336)
3,16760630,/321-585-la-trobe-st-melbourne-vic-3000-16760630,$500/W Furnished,2.0,1.0,NaN,Apartment / Unit / Flat,NaN,"2, ,1, ,,",MELBOURNE,3000,2023-12-02 19:11:26,2023_12,"585 la trobe st, Melbourne, VIC 3000",POINT (144.9517693 -37.8132804)
4,16762357,/1415-325-collins-street-melbourne-vic-3000-16...,$585 per week,2.0,1.0,NaN,Apartment / Unit / Flat,NaN,"2, ,1, ,,",MELBOURNE,3000,2023-12-02 19:11:26,2023_12,"325 collins street, Melbourne, VIC 3000",POINT (144.96353 -37.8168615)


In [23]:
cleaned_df[cleaned_df['coordinates'].isna()]

,property_id,url,rental_price,bedrooms,bathrooms,car_spaces,property_type,land_area,property_features,suburb,postcode,scraped_date,period,address,coordinates
22,16734544,/1-38-lansdowne-street-st-kilda-east-vic-3183-...,$470.00,2.0,1.0,1.0,Apartment / Unit / Flat,NaN,"2, ,1, ,1,",ST KILDA EAST,3183,2023-12-02 22:15:58,2023_12,"38 lansdowne street st kilda, East, VIC 3183",NaN
28,16770376,/6-26-blenheim-street-st-kilda-east-vic-3183-1...,$600 pw,2.0,1.0,1.0,Apartment / Unit / Flat,NaN,"2, ,1, ,1,",ST KILDA EAST,3183,2023-12-02 22:15:58,2023_12,"26 blenheim street st kilda, East, VIC 3183",NaN
34,16766799,/22-woodstock-street-st-kilda-east-vic-3183-16...,$750 per week,2.0,2.0,1.0,Townhouse,NaN,"2, ,2, ,1,",ST KILDA EAST,3183,2023-12-02 22:15:58,2023_12,"woodstock street st kilda, East, VIC 3183",NaN
65,15866775,/rear-70-neil-street-beaufort-vic-3373-15866775,$250.00 per week,2.0,1.0,1.0,Apartment / Unit / Flat,NaN,"2, ,1, ,1,",BEAUFORT,3373,2023-12-07 19:25:48,2023_12,"rear 70 neil street, Beaufort, VIC 3373",NaN
83,16783760,/165-phillips-road-castle-creek-road-wodonga-v...,$650 per week,4.0,2.0,2.0,House,NaN,"4, ,2, ,2,",WODONGA,3690,2023-12-07 20:17:49,2023_12,"phillips road castle creek road, Wodonga, VIC ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11619,16999019,/1006-41-49-bank-street-south-melbourne-vic-32...,$590 per week,1.0,1.0,1.0,Apartment / Unit / Flat,NaN,"1, ,1, ,1,",SOUTH MELBOURNE,3205,2025-07-24 05:50:31,2025_06,"41 49 bank street south, Melbourne, VIC 3205",NaN
11621,17637372,/2-56-smith-street-south-melbourne-vic-3205-17...,$550 per week,2.0,1.0,1.0,Apartment / Unit / Flat,NaN,"2, ,1, ,1,",SOUTH MELBOURNE,3205,2025-07-24 05:50:31,2025_06,"56 smith street south, Melbourne, VIC 3205",NaN
11626,17649466,/g03-127-cardigan-street-carlton-vic-3053-1764...,$1195 per week,2.0,2.0,1.0,Apartment / Unit / Flat,NaN,"2, ,2, ,1,",CARLTON,3053,2025-07-25 08:22:02,2025_06,"g03 127 cardigan street, Carlton, VIC 3053",NaN
11636,17673482,/n2703-570-lygon-street-carlton-vic-3053-17673482,$400 per week,1.0,1.0,NaN,Apartment / Unit / Flat,NaN,"1, ,1, ,,",CARLTON,3053,2025-07-25 08:22:02,2025_06,"n2703 570 lygon street, Carlton, VIC 3053",NaN


In [37]:
# fix addresses below

def fix_address_formatting(address, suburb):
    """
    Fix address formatting by properly handling multi-word suburb names and cleaning street addresses.
    
    The function handles two main issues:
    1. Multi-word suburb names that get split incorrectly in addresses
    2. Street addresses that contain unit numbers, apartment numbers, etc. that should be removed
    
    Parameters:
    -----------
    address : str
        The current address string (e.g., "unit 1 47 53 wyndham st, Shepparton, VIC 3630")
    suburb : str
        The correct suburb name (e.g., "SHEPPARTON")
        
    Returns:
    --------
    str
        Corrected address string (e.g., "53 Wyndham St, Shepparton, VIC 3630")
    """
    
    if pd.isna(address) or pd.isna(suburb) or not isinstance(address, str) or not isinstance(suburb, str):
        return address
    
    # Convert suburb to title case for proper formatting
    suburb_title = suburb.title()
    
    # Split the address by comma to get parts
    address_parts = address.split(',')
    
    if len(address_parts) < 3:
        return address
    
    # Extract street address, suburb part, and state/postcode
    street_address = address_parts[0].strip()
    suburb_part = address_parts[1].strip()
    state_postcode = address_parts[2].strip()
    
    # Clean the street address - extract only street number and street name
    cleaned_street = clean_street_address(street_address)
    
    # Count words in the correct suburb name
    suburb_words = suburb_title.split()
    
    # If suburb has multiple words, we need to reconstruct the address
    if len(suburb_words) > 1:
        # The issue is that the suburb part contains only the last word(s) of the suburb
        # We need to find how many words from the street address belong to the suburb
        
        # For multi-word suburbs, we need to identify which words from the street address
        # actually belong to the suburb name
        
        # Common patterns:
        # - "SOUTH MELBOURNE" -> street address ends with "south", suburb part is "Melbourne"
        # - "ST KILDA EAST" -> street address ends with "st kilda", suburb part is "East"
        
        # Split street address into words
        street_words = street_address.split()
        
        # Find the correct split point
        # We'll try to match the suburb words with the end of street address + suburb part
        combined_text = f"{street_address} {suburb_part}".lower()
        suburb_lower = suburb_title.lower()
        
        # Find where the suburb name starts in the combined text
        suburb_start_idx = combined_text.find(suburb_lower)
        
        if suburb_start_idx != -1:
            # Extract the part before the suburb name
            before_suburb = combined_text[:suburb_start_idx].strip()
            
            # Clean the street part before suburb
            cleaned_before_suburb = clean_street_address(before_suburb)
            
            # Reconstruct the address
            fixed_address = f"{cleaned_before_suburb}, {suburb_title}, {state_postcode}"
            return fixed_address
        else:
            # Fallback: try to match by word count
            # This is a heuristic approach for cases where exact matching fails
            if len(suburb_words) == 2:
                # Two-word suburb (e.g., "SOUTH MELBOURNE")
                # Check if street address ends with the first word of suburb
                if street_words and street_words[-1].lower() == suburb_words[0].lower():
                    # Remove the last word from street address
                    fixed_street = ' '.join(street_words[:-1])
                    cleaned_fixed_street = clean_street_address(fixed_street)
                    fixed_address = f"{cleaned_fixed_street}, {suburb_title}, {state_postcode}"
                    return fixed_address
            elif len(suburb_words) == 3:
                # Three-word suburb (e.g., "ST KILDA EAST")
                # Check if street address ends with the first two words of suburb
                if (len(street_words) >= 2 and 
                    street_words[-2].lower() == suburb_words[0].lower() and
                    street_words[-1].lower() == suburb_words[1].lower()):
                    # Remove the last two words from street address
                    fixed_street = ' '.join(street_words[:-2])
                    cleaned_fixed_street = clean_street_address(fixed_street)
                    fixed_address = f"{cleaned_fixed_street}, {suburb_title}, {state_postcode}"
                    return fixed_address
    
    # If no suburb changes needed, just clean the street address
    fixed_address = f"{cleaned_street}, {suburb_title}, {state_postcode}"
    return fixed_address


def clean_street_address(street_address):
    """
    Clean street address to keep only the first street number and street name.
    
    Removes unit numbers, apartment numbers, and other prefixes while preserving:
    - Only the FIRST street number (including alphanumeric like "12a")
    - Street names
    - Street types (St, Road, Avenue, etc.)
    
    Parameters:
    -----------
    street_address : str
        Raw street address (e.g., "unit 1 47 53 wyndham st")
        
    Returns:
    --------
    str
        Cleaned street address (e.g., "53 Wyndham St")
    """
    
    if not street_address or not isinstance(street_address, str):
        return street_address
    
    # Split into words
    words = street_address.split()
    
    if not words:
        return street_address
    
    # Common prefixes to remove (case insensitive)
    prefixes_to_remove = {
        'unit', 'apt', 'apartment', 'suite', 'level', 'floor', 'shop', 'office',
        'rear', 'front', 'ground', 'basement', 'mezzanine', 'penthouse',
        'villa', 'townhouse', 'house', 'home', 'property', 'building'
    }
    
    # Remove prefixes from the beginning
    cleaned_words = []
    i = 0
    while i < len(words):
        word = words[i].lower()
        
        # Check if this word is a prefix to remove
        if word in prefixes_to_remove:
            # Skip this word and potentially the next one if it's a number
            i += 1
            # If the next word is a number, skip it too
            if i < len(words) and words[i].isdigit():
                i += 1
        else:
            # This is not a prefix, start collecting words
            break
    
    # Collect the remaining words
    cleaned_words = words[i:]
    
    if not cleaned_words:
        return street_address  # Return original if everything was removed
    
    # Find the street number and street name
    # We want the last numeric/alphanumeric word as the street number
    # and everything after it as the street name
    
    street_number = None
    street_name_start = 0
    
    # Look for the last numeric/alphanumeric word in the cleaned words
    for i in range(len(cleaned_words) - 1, -1, -1):
        word = cleaned_words[i]
        # Check if this word contains a number (could be "53", "12a", "415a")
        if any(char.isdigit() for char in word):
            street_number = word
            street_name_start = i + 1
            break
    
    # If no street number found, return the cleaned words as is (title case)
    if street_number is None:
        return ' '.join(word.title() for word in cleaned_words)
    
    # Extract street name (everything after the street number)
    street_name_words = cleaned_words[street_name_start:]
    
    if not street_name_words:
        # If no street name, just return the street number
        return street_number
    
    # Combine street number and street name
    street_name = ' '.join(word.title() for word in street_name_words)
    return f"{street_number} {street_name}"

# Test the function with some examples
test_cases = [
    # Multi-word suburb fixes
    ("56 smith street south, Melbourne, VIC 3205", "SOUTH MELBOURNE"),
    ("38 lansdowne street st kilda, East, VIC 3183", "ST KILDA EAST"),
    ("109 york street south, Melbourne, VIC 3205", "SOUTH MELBOURNE"),
    ("21 lansdowne road st kilda, East, VIC 3183", "ST KILDA EAST"),
    
    # Street address cleaning examples
    ("unit 1 47 53 wyndham st, Shepparton, VIC 3630", "SHEPPARTON"),
    ("415a 72 ascot vale road, Flemington, VIC 3031", "FLEMINGTON"),
    ("evans crescent street, Laverton, VIC 3028", "LAVERTON"),  # No street number
    ("12a nursery street, Croydon, VIC 3136", "CROYDON"),  # Alphanumeric street number
    
    # Combined cases
    ("unit 5 123 smith street south, Melbourne, VIC 3205", "SOUTH MELBOURNE"),
    ("apt 2 45 lansdowne street st kilda, East, VIC 3183", "ST KILDA EAST"),
    
    # Should remain unchanged
    ("33 rose lane, Melbourne, VIC 3000", "MELBOURNE")
]

print("Testing address fixing function:")
print("=" * 80)
for address, suburb in test_cases:
    fixed = fix_address_formatting(address, suburb)
    print(f"Original: {address}")
    print(f"Fixed:    {fixed}")
    print(f"Suburb:   {suburb}")
    print("-" * 80)

Testing address fixing function:
Original: 56 smith street south, Melbourne, VIC 3205
Fixed:    56 Smith Street, South Melbourne, VIC 3205
Suburb:   SOUTH MELBOURNE
--------------------------------------------------------------------------------
Original: 38 lansdowne street st kilda, East, VIC 3183
Fixed:    38 Lansdowne Street, St Kilda East, VIC 3183
Suburb:   ST KILDA EAST
--------------------------------------------------------------------------------
Original: 109 york street south, Melbourne, VIC 3205
Fixed:    109 York Street, South Melbourne, VIC 3205
Suburb:   SOUTH MELBOURNE
--------------------------------------------------------------------------------
Original: 21 lansdowne road st kilda, East, VIC 3183
Fixed:    21 Lansdowne Road, St Kilda East, VIC 3183
Suburb:   ST KILDA EAST
--------------------------------------------------------------------------------
Original: unit 1 47 53 wyndham st, Shepparton, VIC 3630
Fixed:    53 Wyndham St, Shepparton, VIC 3630
Suburb:   SHE

In [39]:
# Apply the fix to the dataframe
print("Fixing addresses in the dataframe...")
print(f"Total records: {len(cleaned_df):,}")

# Apply the fix to addresses
cleaned_df['address_fixed'] = cleaned_df.apply(
    lambda row: fix_address_formatting(row['address'], row['suburb']), 
    axis=1
)

# Check how many addresses were changed
addresses_changed = cleaned_df['address'] != cleaned_df['address_fixed']
print(f"Addresses that were changed: {addresses_changed.sum():,}")

# Show some examples of the changes
if addresses_changed.sum() > 0:
    print("\nExamples of address fixes:")
    print("=" * 80)
    changed_examples = cleaned_df[addresses_changed][['address', 'address_fixed', 'suburb']].head(10)
    for idx, row in changed_examples.iterrows():
        print(f"Original: {row['address']}")
        print(f"Fixed:    {row['address_fixed']}")
        print(f"Suburb:   {row['suburb']}")
        print("-" * 80)

# Update the address column with the fixed addresses
cleaned_df['address'] = cleaned_df['address_fixed']
cleaned_df = cleaned_df.drop(columns=['address_fixed'])

print(f"\nAddress fixing completed!")


Fixing addresses in the dataframe...
Total records: 11,703
Addresses that were changed: 11,668

Examples of address fixes:
Original: 33 rose lane, Melbourne, VIC 3000
Fixed:    33 Rose Lane, Melbourne, VIC 3000
Suburb:   MELBOURNE
--------------------------------------------------------------------------------
Original: 181 exhibition street, Melbourne, VIC 3000
Fixed:    181 Exhibition Street, Melbourne, VIC 3000
Suburb:   MELBOURNE
--------------------------------------------------------------------------------
Original: 225 elizabeth street, Melbourne, VIC 3000
Fixed:    225 Elizabeth Street, Melbourne, VIC 3000
Suburb:   MELBOURNE
--------------------------------------------------------------------------------
Original: 585 la trobe st, Melbourne, VIC 3000
Fixed:    585 La Trobe St, Melbourne, VIC 3000
Suburb:   MELBOURNE
--------------------------------------------------------------------------------
Original: 325 collins street, Melbourne, VIC 3000
Fixed:    325 Collins Street, M

In [40]:
for address_string in cleaned_df[cleaned_df['coordinates'].isna()]['address']:
    print(address_string)

38 Lansdowne Street, St Kilda East, VIC 3183
26 Blenheim Street, St Kilda East, VIC 3183
Woodstock Street, St Kilda East, VIC 3183
Neil Street, Beaufort, VIC 3373
Phillips Road Castle Creek Road, Wodonga, VIC 3690
7 Nelson Street, Ringwood, VIC 3134
12a Nursery Street, Croydon, VIC 3136
4 Wallace Road, Bayswater, VIC 3153
386 Burwood Highway, Burwood, VIC 3125
386 Burwood Highway, Burwood, VIC 3125
386 Burwood Highway, Burwood, VIC 3125
Evans Crescent Street, Laverton, VIC 3028
1330 Dandenong Road, Hughesdale, VIC 3166
Merriang South Road, Myrtleford, VIC 3737
3737, Myrtleford, VIC 3737
72 Ascot Vale Road, Flemington, VIC 3031
27 O Connell Street, North Melbourne, VIC 3051
2 Wests Road, Maribyrnong, VIC 3032
Rattrays Rd, Ondit, VIC 3249
Colac Ballarat Road, Ondit, VIC 3249
Nalangil Road, Nalangil, VIC 3249
1 Rivergum Avenue, Benalla, VIC 3672
Livingston Bvd, Benalla, VIC 3672
Midland Hwy Nalinga, Benalla, VIC 3672
Cultura Walk, Cranbourne West, VIC 3977
Maidenhair Walk, Cranbourne West

In [45]:
# apply the gecoding function to addresses with missing coordinates 
# Fix the pandas warning by using .loc[] for explicit assignment
mask = cleaned_df['coordinates'].isna()
cleaned_df.loc[mask, 'coordinates'] = cleaned_df.loc[mask, 'address'].apply(geoutils.geocode_nominatim)

Status code: 200
Response text: []
Response headers: {'Server': 'nginx', 'Date': 'Thu, 25 Sep 2025 08:23:53 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '2', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=20', 'Vary': 'accept-language'}
Response url: https://nominatim.openstreetmap.org/search?q=38+Lansdowne+Street%2C+St+Kilda+East%2C+VIC+3183%2C+Australia&format=json&limit=1&countrycodes=au&addressdetails=1
Response request: <PreparedRequest [GET]>
Response cookies: <RequestsCookieJar[]>
Response history: []
Response elapsed: 0:00:02.717603
Response raw: <urllib3.response.HTTPResponse object at 0x113a2f940>
No results found for address: 38 Lansdowne Street, St Kilda East, VIC 3183, Australia
Status code: 200
Response text: []
Response headers: {'Server': 'nginx', 'Date': 'Thu, 25 Sep 2025 08:23:56 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '2', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=20', 'Vary': 'accept-langu

In [48]:
cleaned_df['property_id'].nunique()

11703

In [51]:
cleaned_df.shape

(11703, 15)

In [53]:
cleaned_df['property_type'].value_counts()

property_type
House                            6502
Apartment / Unit / Flat          3687
Townhouse                        1304
Studio                             96
Villa                              30
Acreage / Semi-Rural               22
New House & Land                   13
New Apartments / Off the Plan       9
Semi-Detached                       8
Terrace                             8
Duplex                              7
Rural                               4
Unknown                             3
Farm                                3
Block of Units                      3
Vacant land                         2
Penthouse                           1
New land                            1
Name: count, dtype: int64

In [57]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11703 entries, 0 to 11702
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   property_id        11703 non-null  int64  
 1   url                11703 non-null  object 
 2   rental_price       11703 non-null  object 
 3   bedrooms           11631 non-null  float64
 4   bathrooms          11676 non-null  float64
 5   car_spaces         10383 non-null  float64
 6   property_type      11703 non-null  object 
 7   land_area          1011 non-null   float64
 8   property_features  11703 non-null  object 
 9   suburb             11703 non-null  object 
 10  postcode           11703 non-null  int64  
 11  scraped_date       11703 non-null  object 
 12  period             11703 non-null  object 
 13  address            11703 non-null  object 
 14  coordinates        11417 non-null  object 
dtypes: float64(4), int64(2), object(9)
memory usage: 1.3+ MB


In [ ]:
# drop the property_features column 
cleaned_df = cleaned_df.drop(columns=['property_features'])

# rename period column to listing_quarter
cleaned_df = cleaned_df.rename(columns={'period': 'listing_quarter'})




In [59]:
cleaned_df.to_csv('../data/processed/domain/rental_listings_summary_cleaned.csv', index=False)